In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib
from prophet import Prophet


c:\Users\Guillermo\Desktop\Agencia_IA\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [34]:
df = pd.read_csv("../dataset/sales.csv")

In [52]:
df.columns

Index(['fecha', 'ventas_totales', 'num_clientes', 'promociones', 'temporada'], dtype='object')

In [49]:
#entrenamos el modelo prophet

df_prophet = df[['fecha','ventas_totales']].rename(columns={'fecha':'ds','ventas_totales':'y'})

model_prophet = Prophet(yearly_seasonality= True)
model_prophet.fit(df_prophet)

future = model_prophet.make_future_dataframe(periods=12,freq='M')
forecast = model_prophet.predict(future)

forecast.head()

19:10:43 - cmdstanpy - INFO - Chain [1] start processing
19:10:43 - cmdstanpy - INFO - Chain [1] done processing
c:\Users\Guillermo\Desktop\Agencia_IA\venv\Lib\site-packages\prophet\forecaster.py:1854: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(


,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2018-01-01,20446.714210,14767.389229,18309.598296,20446.714210,20446.714210,-3934.719490,-3934.719490,-3934.719490,-3934.719490,-3934.719490,-3934.719490,0.0,0.0,0.0,16511.994720
1,2018-02-01,20448.352722,14353.736182,17992.682343,20448.352722,20448.352722,-4331.140110,-4331.140110,-4331.140110,-4331.140110,-4331.140110,-4331.140110,0.0,0.0,0.0,16117.212612
2,2018-03-01,20449.832668,15495.638121,19037.144635,20449.832668,20449.832668,-3228.727083,-3228.727083,-3228.727083,-3228.727083,-3228.727083,-3228.727083,0.0,0.0,0.0,17221.105585
3,2018-04-01,20451.471179,16602.856202,20073.623793,20451.471179,20451.471179,-2040.679873,-2040.679873,-2040.679873,-2040.679873,-2040.679873,-2040.679873,0.0,0.0,0.0,18410.791306
4,2018-05-01,20453.056835,18020.058245,21539.985513,20453.056835,20453.056835,-732.188544,-732.188544,-732.188544,-732.188544,-732.188544,-732.188544,0.0,0.0,0.0,19720.868291


In [61]:
df_real = df_prophet.tail(12)
y_pred=forecast.tail(12)['yhat']
y_real = df_real['y']
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

mae = mean_absolute_error(y_real, y_pred)
rmse = np.sqrt(mean_squared_error(y_real, y_pred))
r2 = r2_score(y_real, y_pred)

print(mae,rmse,r2)

2056.4480602054787 2176.4394048237687 0.7146319014557123


In [62]:
#entrenamos el modelo de Linear Regression

df['mes_num']=range(len(df))
X = df[['mes_num','num_clientes', 'promociones', 'temporada']]
y = df['ventas_totales']

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42,test_size=0.2)

modelo_linear = LinearRegression()
modelo_linear.fit(X_train,y_train)
prediccion = modelo_linear.predict(X_test)

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

precision = r2_score(y_test,prediccion)
mae= mean_absolute_error(y_test,prediccion)
mse=mean_squared_error(y_test,prediccion)
print(precision,mae,mse)

0.9985084982153478 123.48485998366863 26268.227346283755


In [64]:
#guardamos el modelo de Linaer Regression
joblib.dump(modelo_linear,'modelo_entrenado_prediccion.pkl')


['modelo_entrenado_prediccion.pkl']